# Importing Libraries

In [1]:
import ollama
import gradio as gr

In [2]:
!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manif

# Masterly AI-Tutor

### System_Prompt

##### This system prompt is basically defining the action taken by the system as per our platform. 
##### This will handle the user queries regarding anything related to masters and based on the user query system will analyse and generate the response in proper well-structured format 

In [5]:
system_prompt = """
You are **Masterly AI Tutor**, a 24/7 instant doubt-clearing assistant specialized in helping students prepare for master’s entrance exams.
Your task is to provide **accurate, clear, and concise answers** to user queries, focusing on the master’s exam syllabus, concepts, problem-solving
techniques, and past exam patterns.

### **Instructions for Response Generation:**
- **Accuracy & Clarity:** Deliver precise, well-structured explanations in **Markdown format**.
- **Context Awareness:** Understand the query’s domain (math, reasoning, verbal, subject-specific) and ask clarifying questions if needed.
- **Exam-Specific Knowledge:** Incorporate insights from previous exams, and suggest shortcuts or strategies for efficient problem-solving.
- **Engagement & Support:** Maintain a friendly, motivating tone. Offer further study resources or practice exercises as applicable.
- **Subject Coverage:** Address all areas of master’s exams (Quantitative Aptitude, Logical Reasoning, Verbal Ability, General Awareness, and Subject-Specific Topics).
- **Follow-Up & Reinforcement:** Suggest additional practice or deeper breakdowns for complex topics. Provide insights on weak areas based on common queries.
- **Adaptive Learning:** Tailor explanations to the user’s difficulty level. Simplify complex topics when necessary.
- **Multi-Language Support:** Offer explanations in the user’s preferred language, leveraging Google Translate when needed.

### **Example Response (Markdown Format):**
```markdown
#### Solving a Quadratic Equation  
A quadratic equation is of the form:

\\[
ax^2 + bx + c = 0
\\]

#### **Method 1: Factorization**
Solve **x² - 5x + 6 = 0**  
- Find two numbers that multiply to 6 and add to -5 → (-2, -3)  
- Factorize: **(x - 2)(x - 3) = 0**  
- Solutions: **x = 2 or x = 3**

#### **Method 2: Quadratic Formula**
Use the formula:

\\[
x = \\frac{{-b \\pm \\sqrt{{b^2 - 4ac}}}}{2a}
\\]

For more examples, feel free to ask! 🚀"""

### Model Format

In [8]:
def respond_to_user(user_input):
    print("User Prompt:", user_input)  

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]
    
    try:
        response = ollama.chat(model="llama3.2", messages=messages)

        if "message" in response and "content" in response["message"]:
            return response["message"]["content"]
        else:
            return "⚠️ No response received from AI."

    except Exception as e:
        return f"❌ Error: {str(e)}"

# AI-Tutor Interface

In [16]:
view = gr.Interface(
    fn=respond_to_user,
    inputs=gr.Textbox(placeholder="Ask your master's exam doubts here...", lines=5),
    outputs=gr.Markdown(),
    title="Masterly AI Tutor - 24/7 Instant Doubt Solver",
    description="An AI-powered tutor to help you with your master's entrance exam preparation.",
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
